## Additional metadata support

This notebook takes the output from notebook 1. It iterates through all of the worksheets in all of the Excel files and attempts to pull out all of the unique variable names while storing data about the location of the header rows and data in the database. The unique variables are stores in a table and a join table links them to the unique sheet types. 

The idea is that eventually, each unique sheet type will have a table and the data from the tables that matches those types will be stored in them, regardless of the source. 

After the variables are stored, a manual pass is made through the data to create the normalized column names that will appear in the data tables.

In [1]:
# Manipulate the file system
import os
import shutil
import datetime
import arrow

# Convert stored string representation of a list to a list
import ast

# Recurse through a directory tree and return file names with glob
import glob

# Decode and re-encode mangled Arabic file names
import codecs

# Connect to a SQLite database in a lazy manner.
import dataset

# Enables opening and reading of Excel files
import openpyxl

# Translating variables, sheet names, and workbook names from Arabic
# This is NOT free to use.
from google.cloud import translate

# Set the environment variable for the Google Service Account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:\\Users\\clay\\Documents\\fxb-lcs-2b24f4f8a73a.json'

The sheet names were manually normalized in the database template prior to this step. The template normalization preserves the metadata.

In [2]:
# If there's an existing db for this sheet, delete it
# so that we can copy from the template for a fresh start

try:
    os.remove("sams_data_phase02.sqlite")
    print("Removed template clone sams_data_phase02.sqlite")
except:
    pass

try:
    # Try to preserve a copy in case there is a problem and it has to be restored
    shutil.copy2("sams_data_phase02_template.sqlite","sams_data_phase02.sqlite")
    
    print("Created database from template: sams_data_phase02.sqlite")
except:
    pass

Removed template clone sams_data_phase02.sqlite
Created database from template: sams_data_phase02.sqlite


The prior notebook ends with copying the database to a template file that will again be copied to create the active file for this notebook. That means that you can run all of the cells in this notebook without destroying the database that is required to work with it.

In [3]:
db = dataset.connect("sqlite:///sams_data_phase02.sqlite")

Run a query that pulls out a file path and sheet name for every sheet that we are going to try to parse variables from (that are not marked skip in the sheets table).

In [4]:
recs_to_process = db.query("""
SELECT files_sheets.id AS files_sheets_id, files_sheets.file_id, files_sheets.sheet_id, files.path AS file_path,sheets.name AS sheet_name
FROM files_sheets
JOIN files ON files_sheets.file_id = files.id
JOIN sheets ON files_sheets.sheet_id = sheets.id
WHERE sheets.skip = 0
ORDER BY file_id, sheet_id;
""")

Define a function that will attempt to extract the table headers from the sheets in question.

In [5]:
def headers_from_worksheet(workbook,worksheet_name):
    worksheet = workbook.get_sheet_by_name(worksheet_name)
    winning_row_values = 0
    winning_row_number = None
    
    for row in range(1,21):
        start_range = 'A' + str(row)
        end_range = 'Z' + str(row)
        cells = worksheet[start_range:end_range]
        try:
            values = [c.value for c in cells[0]]
        except:
            continue
            
        filled_cells = 0
        for v in values:
            if v is not None:
                filled_cells += 1
        
        if filled_cells > winning_row_values:
            winning_row_values = filled_cells
            winning_row_number = row
            
    # Some sheets may be blank
    if winning_row_number is None:
        return None,None
            
    winning_start = 'A' + str(winning_row_number)
    winning_end = 'Z' + str(winning_row_number)

    header_cells = worksheet[winning_start:winning_end]
    header_data = [c.value for c in header_cells[0]]
    
    # If we detect a datetime.datetime.object, then we probably
    # want the previous row. Might be a better way to check this
    # TODO: This is a bad idea?
    
    # What's the actual start column of the header?
    start_idx = 0
    determined_start = False
    
    for val in header_data:
        if not determined_start and val is None:
            start_idx += 1
        elif val is not None:
            determined_start = True
            
        if isinstance(val,datetime.datetime):
            winning_row_number -= 1
            
    header_start_letter = letter_lookup[start_idx]
            
    winning_start = header_start_letter + str(winning_row_number)
    winning_end = 'Z' + str(winning_row_number)
    try:
        header_cells = worksheet[winning_start:winning_end]
        header_data = [c.value for c in header_cells[0]]
    except:
        return None,None
    
    end_idx = len(header_data) - 1
    problem = ''
#     print(end_idx)
#     print(header_data)
#     print(header_data[end_idx],"\n")
    
    while header_data[end_idx] is None:
        end_idx -= 1
        if end_idx <= start_idx:
            problem = ' (PROBLEM)'
            break
            
    # Lookup assumes that the header starts with col A, so offset the lookup on the
    # end letter by the start letter index and it will assign the proper letter to the
    # end letter.
    end_letter = letter_lookup[end_idx+start_idx]  
    header_end = end_letter + winning_start[1:] + problem
    header_range = (winning_start,header_end)
    
    # Prune the header_data to get rid of trailing None values
    prune_by = 0
    
    while header_data[prune_by-1] is None:
        prune_by -= 1
        
    try:
        header_data = header_data[:prune_by]
    except:
        pass
    
    return header_range, header_data

In [6]:
tab_files_sheets = db['files_sheets']

In [7]:
letter_lookup = ['A','B','C','D','E','F','G','H','I','J','K',
                 'L','M','N','O','P','Q','R','S','T','U','V',
                 'W','X','Y','Z']

In [12]:
variables = set()

working_file_id = -1
active_file_path = None
active_workbook = None

for rec in recs_to_process:
    
    # This only fires with a new file_id
    if rec['file_id'] > working_file_id:
        working_file_id = rec['file_id']
        active_file_path = rec['file_path']
        try:
            active_workbook = openpyxl.load_workbook(active_file_path,read_only=True,guess_types=False,data_only=True)
        except:
            print("Unable to open",active_file_path)
            active_workbook = None
            active_file_path = None
            working_file_id = -1
            
    # Process the active file
    sheet_name = rec['sheet_name']
    header_range, header_data = headers_from_worksheet(active_workbook,sheet_name)
    
    # Unable to find a header in this sheet. Mark the record
    if header_range is None:
        update_rec = {"id":rec['files_sheets_id'],"header_start":"PROBLEM"}
        tab_files_sheets.update(update_rec,['id'])
        print(rec['files_sheets_id'],"Problem workbook",active_file_path,"--> sheet -->",sheet_name)
        continue
    else:
        header_start = header_range[0]
        header_end = header_range[1]
        
        fixed_header_data = []
        for value in header_data:
            if isinstance(value,datetime.datetime):
                fixed_value = arrow.get(value).format("YYYY-MM-DD")
                fixed_header_data.append(fixed_value)
                variables.add(fixed_value)
            else:
                fixed_header_data.append(value)
                variables.add(value)
        
        update_rec = {
            "id":rec['files_sheets_id'],
            "header_start":header_start,
            "header_end":header_end,
            "header_values":str(fixed_header_data)
        }
        
        tab_files_sheets.update(update_rec,['id'])
        

907 Problem workbook data\jordan\2016\April\π∩ƒºƒó δφε ¼∞⌐ 4-2016.xlsx --> sheet --> وصفات شهرية
1677 Problem workbook data\jordan\2016\Feb\½ñΘ Ω⌐Φ¬ π∩ƒºƒó ñƒ½Ω Θ¼∞⌐ 2-2016.xlsx --> sheet --> الفيزيائي
5390 Problem workbook data\turkey\2015\Feb\Jisr Al shgur\Dental and Dermatological\Patient log 1.xlsx --> sheet --> ك


Create a table to hold the unique variables collected and translate them with the Google Translate API.

In [13]:
translate_client = translate.Client()
target_lang = 'en'

In [14]:
try:
    tab_vars.drop()
except:
    pass

tab_vars = db['variables']

In [15]:
for v in variables:
    try:
        v_str = str(v)
        clean = v_str.replace("\n"," ").replace("\\","").replace("\t"," ").strip()
    except:
        print("Could not process",v)
        continue
        
    translation = translate_client.translate(v_str,target_language=target_lang)
        
    rec = {
        "orig":v,
        "clean":clean,
        "translation":translation['translatedText'],
        "normalized":""
    }
    try:
        tab_vars.insert(rec)
    except:
        print("\nFailure to insert")
        print(rec)

In [16]:
# FIRST PASS AT VARIABLES IS IN THE DB
# - Find names and figure out which sheets are importing wrong.
# - Manually set the proper header ranges?

Iterate through the files_sheets, and create reference tables that connect the sheets table and the files_sheets table to the variables. This is mainly for diagnostic purposes and for debugging which sheets and files have header reference problems.

In [17]:
try:
    tab_files_vars.drop()
    tab_sheets_vars.drop()
    tab_files_sheets_vars.drop()
except:
    pass

tab_files_vars = db['files_variables']
tab_sheets_vars = db['sheets_variables']
tab_files_sheets_vars = db['files_sheets_vars']

Hold a variable reference in memory first.

In [18]:
var_lookup = {}
for rec in tab_vars.find():
    var_lookup[rec['orig']] = rec['id']

In [19]:
files_vars_set = set()
sheets_vars_set = set()
files_sheets_vars_set = set()

for rec in tab_files_sheets.find():
    header_values = rec['header_values']
    if header_values is None:
        continue
    
    header_values = ast.literal_eval(rec['header_values'])
    
    for header in header_values:
        if header is None:
            continue
        try:
            var_id = var_lookup[str(header)]
            file_id = rec['file_id']
            sheet_id = rec['sheet_id']
            files_sheets_id = rec['id']
        except:
            print("problem with",header)
            continue
        
        files_vars_set.add((file_id,var_id))
        files_sheets_vars_set.add((files_sheets_id,var_id))
        sheets_vars_set.add((sheet_id,var_id))

Create the join tables that we'll use for research into problems.

In [20]:
tab_files_vars_recs = []

for rec_tuple in files_vars_set:
    rec = {"file_id":rec_tuple[0],"var_id":rec_tuple[1]}
    tab_files_vars_recs.append(rec)
    
tab_files_vars.insert_many(tab_files_vars_recs)

In [21]:
tab_sheets_vars_recs = []

for rec_tuple in sheets_vars_set:
    rec = {"sheet_id":rec_tuple[0],"var_id":rec_tuple[1]}
    tab_sheets_vars_recs.append(rec)
    
tab_sheets_vars.insert_many(tab_sheets_vars_recs)

In [22]:
tab_files_sheets_vars_recs = []

for rec_tuple in files_sheets_vars_set:
    rec = {"files_sheets_id":rec_tuple[0],"var_id":rec_tuple[1]}
    tab_files_sheets_vars_recs.append(rec)
    
tab_files_sheets_vars.insert_many(tab_files_sheets_vars_recs)

Up to this point, we've tried to guess what the header row is for each of the sheets in each of the Excel files. There are errors, however, and they have to be fixed manually.

The process is to identify the most problematic header indexing errors through querying unusual variable names and tracing them back to the `files_sheets` entry that corresponds. The process of identifying the problems can be handled in two ways: the first is that some of the problems are systematic and the header extraction code can be updated to pull those schema in a more appropriate manner and the second is manual, where for any number of reasons, we either have columns that do not belong, columns that are not named properly or have a generic name, or sheets that were unparseable. 

Some of the unparseable sheets are simply empty and should be skipped, some of them contain aggregate data, which we currently are not importing, some come from corrupt files, and some are a problem with the header extraction algorithm.

This step seeks to manually repair the largest header range issues in the `files_sheets` page and the to re-extract the header values from those sheets using the references instead of the generic algorithm. That will trigger another variable review. Iterate this process until the error is tolerable and then this step of the process is complete.

In [23]:
# There are sheets that have header references but are blank.
# blank them out in the database.
recs = tab_files_sheets.find()

update_recs = []

for rec in recs:
    if rec['header_values'] == '[]':
        ur = {
            "id":rec['id'],
            "header_start":None,
            "header_end":None,
            "header_values":None
        }
        update_recs.append(ur)
        
for rec in update_recs:
    tab_files_sheets.update(rec,['id'])

Due to the manual effort that has gone into cleaning files_sheets, this notebook will conclude by saving out the current database as the template for the next database.

In [24]:
# import shutil

In [25]:
# Do not rerun this cell!
shutil.copy2('sams_data_phase02.sqlite','sams_data_phase03_template.sqlite')

'sams_data_phase03_template.sqlite'